# Livestock literature analysis

In [ ]:
%%capture
!pip install netcdf4 metpy colorcet
import os, fnmatch, gzip, shutil, numpy as np, pandas as pd, xarray as xr
import pylab as plt, plotly.express as px, seaborn as sns, colorcet as cc, altair as alt
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
import cv2; from google.colab.patches import cv2_imshow
%load_ext google.colab.data_table
from metpy.units import units; import metpy.calc as mpcalc

!pip install --upgrade zarr gcsfs cftime nc-time-axis
import fsspec, zarr, gcsfs; gcs = gcsfs.GCSFileSystem( token = 'anon' )

#!apt-get -qq install libgeos-dev
#!pip install -qq https://github.com/matplotlib/basemap/archive/master.zip
#from mpl_toolkits.basemap import Basemap, maskoceans

def rgb2hex( rgbstring ):
    s = rgbstring.split('(')[1].split(',')
    return "#{0:02x}{1:02x}{2:02x}".format(max(0, min(np.int(s[0]), 255)), max(0, min(np.int(s[1]), 255)), max(0, min(np.int(s[2].split(')')[0]), 255)))

def hex2rgb( value ):
    value = value.lstrip('#')
    return tuple(int(value[i:i + len(value) // 3], 16) for i in range(0, len(value), len(value) // 3))

from sklearn.neighbors import KernelDensity
def skkde( x, x_grid, bandwidth = 0.2, **kwargs ):
    kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
    kde_skl.fit(x[:, np.newaxis])
    log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
    return np.exp(log_pdf)

from google.colab import drive; drive.mount( '/content/drive' )

### Ruminant THIs

In [ ]:
def THI(spec, Ta, Tdp = False, huss = False):
    # takes dry-bulb temperature in C and either dewpoint (C) or specific humidity (kg/kg)
    # returns THI based on whichever specificciation you choose
    Ta = np.nan_to_num( np.atleast_1d( Ta ) )
    temp  = Ta * units.degC
    press = np.array( 1e5 ) * units.hectopascal

    if huss == False:
        Tdp = np.nan_to_num( np.atleast_1d( Tdp ) )
        dew = Tdp * units.degC
        RH  = np.asarray( mpcalc.relative_humidity_from_dewpoint( temp , dew ) )
        ### Wet bulb from Stull, 2011 Journal of Applied Meterology and Climatology, Equation 1
        Twb = Ta * np.arctan( 0.151977 * (RH + 8.313659)**(1/2) ) + np.arctan(Ta+ RH) - np.arctan(RH - 1.676331) + 0.00391838 * (RH)**(3/2) * np.arctan(0.023101 * RH) - 4.686035

    elif Tdp == False:
        huss = np.nan_to_num( np.atleast_1d( huss ) )
        SH  =  huss * units.dimensionless
        RH  = np.asarray( mpcalc.relative_humidity_from_specific_humidity(SH, temp, press ) )
        Tdp = np.asarray( mpcalc.dewpoint_from_specific_humidity( SH, temp, press ) )

    if   spec == 'thom':   thi = 0.80*Ta + Ta*RH - 1.43*RH + 46.4     ## adapted from Thom59, they do something goofy with the conversion
    elif spec == 'kibler': thi = 1.00*Ta + 0.36*Tdp + 41.2            ## Kibler 64, NRC71, then yousef85
    elif spec == 'berman': thi = 1.06*Ta + 1.6*Ta*RH - 29*RH + 39.1   ## from berman 2016, not sure if its used anywhere
    elif spec == 'kelly':  thi = Ta - 0.55 * (1 -  RH) * (Ta - 14.5)  ## From kelly and bond 1971, but they use C and F incorectly, so values are lower

    thi = np.ma.masked_outside( np.nan_to_num( thi ), 1, 200 ).compressed()

    return np.round(thi,2)


In [ ]:
THI('kibler', 28, Tdp=10)

In [ ]:
hahn99 == kiebler64 == nrc71 == thom59

### bad eqs

def kelly71(T, Twb):   return T - 0.55 * (1 - 0.01 * RH) * (T - 14.5) # must be F or something
def roditan96(T, RH): return (0.4 * (T + Twb) + 4.8) ## much lower than all the others

def thom59(Ta, Twb):    return 0.72*Ta + 0.72*Twb + 46.4 ###i think this may be badly attributed to Thom

def nrc71(Ta, Twb):     return 0.72*Ta + 0.72*Twb + 40.6 ### almost the same as yousef, i think from kibler64
def nrc71a(Ta, Tdp):    return 1.00*Ta + 0.36*Tdp + 49.5 ### only used for buffalo

old eqs, no impacts:
def kibler64(Ta, RH):   return 0.72*Ta + 0.72*Twb + 40.6

def bianca62a(Ta, Twb): return 0.63*Ta + 1.15*Twb + 32
def bianca62b(Ta, Twb): return 0.45*Ta + 1.35*Twb + 32

### pigs
def ingram65(Ta, Twb):  return 1.17*Ta + 0.63*Twb + 32
def roller69(Ta, Twb):  return 1.35*Ta + 0.45*Twb + 32
def rothfusz90(Ta, RH): return 2.049 * (1.8 * Ta + 32) + (10.143 * RH) - (0.224 * (Ta * 1.8 + 32) * RH) - (0.0068 * ((Ta * 1.8 + 32)**2)) - (0.055 * (RH**2)) + (0.0012 * ((Ta * 1.8 + 32)**2) * RH) + (0.0009 * (Ta * 1.8 + 32) * (RH**2)) - (0.00000199 * ((Ta * 1.8 + 32)**2)*(RH**2)) - 42.379

In [ ]:
### ruminants ###
ax = plt.subplot( gs[0,1] )
THI_r = np.zeros((50,50))
for i, T_max in enumerate(np.linspace(20,50,50)):
    for j, RH in enumerate(np.linspace(0,100,50)):
        THI_r[j,i] = THIr(T_max, RH)

plt.contour( np.linspace(20,50,50), np.linspace(0,100,50), THI_r, levels = [74,84], colors = [linecolor] )
plt.contourf( np.linspace(20,50,50), np.linspace(0,100,50), THI_r, levels = [78, 84], colors = [linecolor], alpha = 0.1 )
plt.annotate('b | Ruminants', xy = (0.63,0.93), xycoords='axes fraction')
#plt.annotate('Des Moines (2000-10)', xy = (0.48,0.81), xycoords='axes fraction', color = '#3690c0')
#plt.annotate('Johannesburg (2000-10)', xy = (0.43,0.87), xycoords='axes fraction', color = '#8c6bb1')
plt.annotate('New Delhi (2000-10)', xy = (0.49,0.75), xycoords='axes fraction', color = '#df65b0')
plt.annotate('Severe', xy = (0.62, 0.01), xycoords='axes fraction', color = '#4d4d4d').set_rotation(-42)
plt.annotate('Emergency', xy = (0.7,0.2), xycoords='axes fraction')
plt.annotate('Comfort', xy = (0.2,0.03), xycoords='axes fraction')
plt.arrow(42, 16, 5, 0, head_width = 1.5, head_length = 1.5, color='black')
plt.arrow(31, 10, -5, 0, head_width = 1.5, head_length = 1.5, color='black')
plt.xticks([20,25,30,35,40,45], [])
plt.yticks([0,25,50,75,100], [])
plt.xlim(20,50)
plt.ylim(0,100)

In [ ]:
import os, fnmatch, gzip, shutil
from google.colab import drive; drive.mount( '/content/drive' )

In [ ]:
root = 'https://www.metoffice.gov.uk/hadobs/hadisd/v311_202008p/data/'

tags = ['WMO_000000-029999_heat_stress.tar.gz','WMO_030000-049999_heat_stress.tar.gz','WMO_050000-079999_heat_stress.tar.gz',
        'WMO_080000-099999_heat_stress.tar.gz','WMO_100000-149999_heat_stress.tar.gz','WMO_150000-199999_heat_stress.tar.gz',
        'WMO_200000-249999_heat_stress.tar.gz','WMO_250000-299999_heat_stress.tar.gz','WMO_300000-349999_heat_stress.tar.gz',
        'WMO_350000-399999_heat_stress.tar.gz','WMO_400000-449999_heat_stress.tar.gz','WMO_450000-499999_heat_stress.tar.gz',
        'WMO_500000-549999_heat_stress.tar.gz','WMO_550000-599999_heat_stress.tar.gz','WMO_600000-649999_heat_stress.tar.gz',
        'WMO_650000-699999_heat_stress.tar.gz','WMO_700000-709999_heat_stress.tar.gz','WMO_710000-719999_heat_stress.tar.gz',
        'WMO_720000-722999_heat_stress.tar.gz','WMO_723000-723999_heat_stress.tar.gz','WMO_724000-724999_heat_stress.tar.gz',
        'WMO_725000-725999_heat_stress.tar.gz','WMO_726000-726999_heat_stress.tar.gz','WMO_727000-729999_heat_stress.tar.gz',
        'WMO_730000-799999_heat_stress.tar.gz','WMO_800000-849999_heat_stress.tar.gz','WMO_850000-899999_heat_stress.tar.gz',
        'WMO_900000-949999_heat_stress.tar.gz','WMO_950000-999999_heat_stress.tar.gz']

tags = [ root + tag for tag in tags ]

#for tag in tags:

In [ ]:
os.system(f'wget {tags[0]}')

In [ ]:
f = gzip.open( '/content/WMO_000000-029999_heat_stress.tar.gz','rb')
file_content = f.read()

In [ ]:
file_content

In [ ]:
with gzip.open('file.txt.gz', 'rb') as f_in:
    with open('file.nc', 'wb') as f_out:
        shutil.copyfileobj(f_in, f'/content/drive/My Drive/data/livestock/HadISD/{f_out}')

In [ ]:
sta = pd.read_fwf('/content/drive/My Drive/data/livestock/hadisd_station_fullinfo_v311_202001p.txt',
                  colspecs = [(0,11),(12,42),(43,50),(51,59),(60,67),(68,78),(79,89)],
                  names = ['ID','name','lat','lon','elv','start','end'])
sta[ sta.name.str.contains('NAIR') ]

In [ ]:
new = sta[sta.lat>49]
new = new[new.lon<-80].sort_values('lat')
new

In [ ]:
newdelhi  = '421820-99999'
desmoines = '725460-14933'
paris = '071560-99999'
london = '037780-99999'
santarosa = '876230-99999'
edmondton = '711210-99999'
darwin = '941200-99999'
douala = '649100-99999'
portgentil = '645010-99999'
nairobi = '637400-99999'

In [ ]:
plt.rcParams.update( {'font.size': 7, 'xtick.labelsize' : 7, 'ytick.labelsize' : 7, 'legend.fontsize': 7, 'legend.frameon': False,
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1.4, 'ytick.major.size':1.4,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':1, 'ytick.major.pad':1, 'axes.labelpad': 0,'xtick.direction': 'out', 'ytick.direction': 'out',
                      'axes.edgecolor': 'black','ytick.color': 'black','xtick.color': 'black'})

plt.close('all')
fig = plt.figure( figsize = ( 5, 4 ) )

colors = px.colors.sequential.Plotly3
x_grid = np.arange(1,120,1)

ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{edmondton}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))
t = 1.8 * ( 0.55 * ds.temperatures.values + 0.2 * ds.dewpoints.values )  + 49.5
t = np.ma.masked_outside( np.nan_to_num(t), 0, 200 ).compressed()
pdf = skkde( t, x_grid, bandwidth = 3 )
plt.plot( x_grid, pdf, color = colors[0], lw = 1 )

ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{london}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))

t = 1.8 * ( 0.55 * ds.temperatures.values + 0.2 * ds.dewpoints.values )  + 49.5
t = np.ma.masked_outside( np.nan_to_num(t), 1, 200 ).compressed()
pdf = skkde( t, x_grid, bandwidth = 3 )
plt.plot( x_grid, pdf, color = colors[3], lw = 1 )

ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{santarosa}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))
t = 1.8 * ( 0.55 * ds.temperatures.values + 0.2 * ds.dewpoints.values )  + 49.5
t = np.ma.masked_outside( np.nan_to_num(t), 1, 200 ).compressed()
pdf = skkde( t, x_grid, bandwidth = 3 )
plt.plot( x_grid, pdf, color = colors[6], lw = 1 )

ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{nairobi}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))
t = 1.8 * ( 0.55 * ds.temperatures.values + 0.2 * ds.dewpoints.values )  + 49.5
t = np.ma.masked_outside( np.nan_to_num(t), 1, 200 ).compressed()
pdf = skkde( t, x_grid, bandwidth = 3 )
plt.plot( x_grid, pdf, color = colors[9], lw = 1 )

ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{newdelhi}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))
t = 1.8 * ( 0.55 * ds.temperatures.values + 0.2 * ds.dewpoints.values )  + 49.5
t = np.ma.masked_outside( np.nan_to_num(t), 1, 200 ).compressed()
pdf = skkde( t, x_grid, bandwidth = 3 )
plt.plot( x_grid, pdf, color = colors[12], lw = 1 )

plt.xticks( [0, 20, 40, 60, 80, 100, 120], [0, 20, 40, 60, 80, 100, 120] )
plt.xlabel('THI')
sns.despine( trim = True, offset = 2 )

plt.savefig( '/content/drive/My Drive/figures/livestock/select_city_thi.png', dpi = 700, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{nairobi}.nc')
ds = ds.sel(time = slice('1975-01-01T12:00:00', '2014-12-30T12:00:00'))
thi = THI( ds.temperatures.values, ds.dewpoints.values, False )

quants = np.quantile(thi, np.arange( 0.01, 1, 0.01 ), axis = 0)

plt.rcParams.update( {'font.size': 7, 'xtick.labelsize' : 7, 'ytick.labelsize' : 7, 'legend.fontsize': 7, 'legend.frameon': False,
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1.4, 'ytick.major.size':1.4,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':1, 'ytick.major.pad':1, 'axes.labelpad': 0,'xtick.direction': 'out', 'ytick.direction': 'out',
                      'axes.edgecolor': 'black','ytick.color': 'black','xtick.color': 'black'})

plt.close('all')
fig = plt.figure( figsize = ( 5, 5 ) )
label = ['bianca62a', 'bianca62b', 'kibler64', 'nrc71', 'yousef85', 'berman16']
for i in range(6): plt.plot( quants[:,i], label = label[i]  )
plt.xlabel( 'Percentile' )
plt.ylabel( 'THI' )
plt.yticks( [20,40,60,80,100], [20,40,60,80,100] )
sns.despine( trim = True, offset = 2 )
plt.legend()

plt.savefig( '/content/drive/My Drive/figures/livestock/new_delhi_this.png', dpi = 700, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
quants.shape

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory = False )
df = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day'" )
df.institution_id.unique()

In [ ]:
lat = 28.2
lon = 77.3
model = 'IPSL-CM6A-LR'
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory = False )

dfs = df.query( f"activity_id=='CMIP' & variable_id == 'tasmax' & experiment_id == 'historical' & source_id == '{model}' & table_id == 'day'" )
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
tasmaxh = ds.sel( lat=lat,  method = 'nearest' ).sel( lon=lon, method='nearest' ).sel( time=slice('1985-01-01T12:00:00','2014-12-31T12:00:00') )

dfs = df.query( f"activity_id=='CMIP' & variable_id == 'hur' & experiment_id == 'historical' & source_id == '{model}' & table_id == 'day'" )
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
hurh = ds.sel( lat=lat,  method='nearest' ).sel( lon=lon, method='nearest' ).sel( plev=1e5 ).sel( time=slice('1985-01-01T12:00:00','2014-12-31T12:00:00') )

dfs = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tasmax' & experiment_id == 'ssp585' & source_id == '{model}' & table_id == 'day'" )
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
tasmax = ds.sel( lat=lat,  method = 'nearest' ).sel( lon=lon, method='nearest' ).sel( time=slice('2070-01-01T12:00:00','2099-12-31T12:00:00') )

dfs = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'hur' & experiment_id == 'ssp585' & source_id == '{model}' & table_id == 'day'" )
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
hur = ds.sel( lat=lat,  method='nearest' ).sel( lon=lon, method='nearest' ).sel( plev=1e5 ).sel( time=slice('2070-01-01T12:00:00','2099-12-31T12:00:00') )

In [ ]:
model = 'IPSL-CM6A-LR'
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory = False )
dfs = df.query( f"variable_id == 'huss' & member_id == 'r1i1p1f1' & experiment_id == 'historical' & table_id == 'day'" )
dfs

In [ ]:
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
ds

In [ ]:
ds

In [ ]:
model = 'IPSL-CM6A-LR'
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory = False )
dfs = df.query( f"activity_id=='CMIP' & variable_id == 'hus' & experiment_id == 'historical' & source_id == '{model}' & table_id == 'day'" )
ds = xr.open_zarr( gcs.get_mapper( dfs.zstore.values[0] ), consolidated = True )
hurh = ds.sel( lat=lat,  method='nearest' ).sel( lon=lon, method='nearest' ).sel( plev=100000.0 , method='nearest' ).sel( time=slice('1985-01-01T12:00:00','2014-12-31T12:00:00') )
x = hurh.hus.values
plt.hist(x)

In [ ]:
ds = xr.open_dataset(f'/content/drive/My Drive/data/livestock/HadISD/hadisd.3.1.1.202007p_19310101-20200801_{newdelhi}.nc')
ds = ds.sel(time = slice('1985-01-01T12:00:00', '2014-12-30T12:00:00'))
Tdp = ds.dewpoints.values
Ta = ds.temperatures.values
RH  = 100*( np.exp( ( 17.625 * Tdp ) / ( 243.04 + Tdp ) ) / np.exp( ( 17.625 * Ta ) / ( 243.04 + Ta ) ) )
RH[ RH < 0 ] = 0
RH[ RH > 100 ] = 100
plt.hist(RH)

In [ ]:
px.colors.qualitative.swatches()

In [ ]:
for tag in tags: os.system(f'wget {tag}')

In [ ]:
f = gzip.open( '/content/WMO_000000-029999_heat_stress.tar.gz','rb')
file_content = f.read()

In [ ]:
file_content

In [ ]:
plt.scatter( np.arange(14), THI(30,10) )
plt.xlabel('order of publication -->')
plt.ylabel('THI (30C, 10C Wetbulb)')
plt.savefig('test.png')

In [ ]:
plt.hist( THI(30,10) )

# Figure 1 - MAP

In [ ]:
img = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
area = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/8_Areakm.tif', cv2.IMREAD_UNCHANGED)
img = img / area
img = np.flip( img, axis = 0 )
img = np.ma.masked_where( img < 0, img )

In [ ]:
plt.close('all')
fig = plt.figure( figsize = ( 10, 5 ) )

colors = [rgb2hex(i) for i in px.colors.sequential.gray_r][1:]
levels = np.linspace( 1, 100, 11 )
img[ img > 102 ] = 102
cmap, norm = flc( levels = levels, colors = colors, extend = 'max')

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update( wspace = 0, hspace = 0.01)

m = Basemap( projection = 'eck4', lon_0 = 0, resolution = 'l' )
lat = np.linspace( -90, 90, img.shape[0] )
lon = np.linspace( -180, 180, img.shape[1] )
lon2d, lat2d = np.meshgrid( lon, lat )
x, y = m( lon2d, lat2d )

ax1 = plt.subplot( gs[0, 0:3] )
m.drawmapboundary( fill_color = 'white', color = 'white', linewidth = 0.0 )
m.pcolormesh( x, y, img, cmap = cmap, zorder = 10, alpha = 0.7)

scatcolors = [rgb2hex(i) for i in px.colors.qualitative.Antique]
scatcolors = px.colors.qualitative.Plotly
for i, animal in enumerate(animals):
    df1 = df[df.Species == animal]
    xa, ya = m( df1.lon.values, df1.lat.values )
    plt.scatter( xa, ya, s = 20, facecolors = scatcolors[i], edgecolors = 'white', lw = 0.3, zorder = 20, alpha = 0.8 )

ax2 = plt.subplot( gs[1, 1] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label( 'Number of animals per km$^2$', fontsize = 7 )

plt.savefig( '/content/drive/My Drive/figures/livestock/cow_density.png', dpi = 700, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
bad_labels = {'water buffalo':'buffalo',
              'Murrah buffaloes':'buffalo',
              'Buffalo':'buffalo',
              'buffaloes':'buffalo',
              'cattle and buffalo':'buffalo',
              'goat':'goats',
              'Sheep':'sheep',
              'sheep & camels':'sheep',
              'cattle ':'cattle',
              'chicken':'chickens' }

df = pd.read_csv('/content/drive/My Drive/data/livestock_sources - literature.csv')
df = df[['Pub year', 'lat', 'lon','Species']].dropna()
df = df.rename(columns={'Pub year': 'year' })
df = df.replace( bad_labels )
df = df.replace( 71, -71 )

df = df[df.Species != 'rabbits']
df = df[df.Species != 'people']

animals = ['cattle', 'buffalo', 'chickens', 'goats', 'pigs', 'sheep' ]

np.random.seed(1)
df['lat'].where(df.duplicated( ['lat']), lambda x: x+np.random.randn()/0.8, inplace=True, axis=0)
df['lon'].where(df.duplicated( ['lon']), lambda x: x+np.random.randn()/0.8, inplace=True, axis=0)
do = pd.DataFrame({'Species':['buffalo', 'chickens', 'cattle', 'goats', 'pigs', 'sheep'], 'order': [2, 6, 1, 4, 3, 5]})
df = pd.merge( df, do, on = ['Species'] )

In [ ]:
plt.close('all')
fig = plt.figure( figsize = ( 10, 5 ) )

colors = [rgb2hex(i) for i in px.colors.sequential.gray_r][1:]
levels = np.linspace( 1, 100, 11 )
img[ img > 102 ] = 102
cmap, norm = flc( levels = levels, colors = colors, extend = 'max')

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update( wspace = 0, hspace = 0.01)

m = Basemap( projection = 'eck4', lon_0 = 0, resolution = 'l' )
lat = np.linspace( -90, 90, img.shape[0] )
lon = np.linspace( -180, 180, img.shape[1] )
lon2d, lat2d = np.meshgrid( lon, lat )
x, y = m( lon2d, lat2d )

ax1 = plt.subplot( gs[0, 0:3] )
m.drawmapboundary( fill_color = 'white', color = 'white', linewidth = 0.0 )
m.pcolormesh( x, y, img, cmap = cmap, zorder = 10, alpha = 0.7)

scatcolors = [rgb2hex(i) for i in px.colors.qualitative.Antique]
scatcolors = px.colors.qualitative.Plotly
for i, animal in enumerate(animals):
    df1 = df[df.Species == animal]
    xa, ya = m( df1.lon.values, df1.lat.values )
    plt.scatter( xa, ya, s = 20, facecolors = scatcolors[i], edgecolors = 'white', lw = 0.3, zorder = 20, alpha = 0.8 )

ax2 = plt.subplot( gs[1, 1] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label( 'Number of animals per km$^2$', fontsize = 7 )

plt.savefig( '/content/drive/My Drive/figures/livestock/cow_density.png', dpi = 700, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
colors = px.colors.qualitative.Antique

def recolor(im, rgbstring):
    s = rgbstring.split('(')[1].split(',')
    imred = im[:,:,2]
    imgreen = im[:,:,1]
    imblue = im[:,:,0]
    imred[imred<255] = np.int(s[0])
    imgreen[imgreen<255] = np.int(s[1])
    imblue[imblue<255] = np.int(s[2].split(')')[0])
    im = np.concatenate( ( imred[:,:,np.newaxis], imgreen[:,:,np.newaxis], imblue[:,:,np.newaxis]), axis = 2 )
    return im

colors = px.colors.qualitative.Plotly
def recolor(im, hex):
    rgb = hex2rgb(hex)
    imred = im[:,:,2]
    imgreen = im[:,:,1]
    imblue = im[:,:,0]
    imred[imred<255] = rgb[0]
    imgreen[imgreen<255] = rgb[1]
    imblue[imblue<255] = rgb[2]
    im = np.concatenate( ( imred[:,:,np.newaxis], imgreen[:,:,np.newaxis], imblue[:,:,np.newaxis]), axis = 2 )
    return im

im = cv2.imread('/content/drive/My Drive/figures/livestock/cow.png', cv2.IMREAD_COLOR)
cow = recolor( im, colors[0] )
im = cv2.imread('/content/drive/My Drive/figures/livestock/buffalo.png', cv2.IMREAD_COLOR)
buffalo = recolor( im, colors[1] )
im = cv2.imread('/content/drive/My Drive/figures/livestock/chicken.png', cv2.IMREAD_COLOR)
chicken = recolor( im, colors[2] )
im = cv2.imread('/content/drive/My Drive/figures/livestock/goat.png', cv2.IMREAD_COLOR)
goat = recolor( im, colors[3] )
im = cv2.imread('/content/drive/My Drive/figures/livestock/pig.png', cv2.IMREAD_COLOR)
pig = recolor( im, colors[4] )
im = cv2.imread('/content/drive/My Drive/figures/livestock/sheep.png', cv2.IMREAD_COLOR)
sheep = recolor( im, colors[5] )

## Isotype plot

In [ ]:
plt.close('all')
fig = plt.figure( figsize = ( 10, 5 ) )
gs = GridSpec(29, 31, wspace = 0.02, hspace = 0.02 )

for i, year in enumerate(range(1990,2021)):
    if i == 29: print(i)
    df1 = df[df.year == year].sort_values('order')
    r = 0
    for index, row in df1.iterrows():
        if row.Species == 'cattle': x = cow
        elif  row.Species == 'buffalo': x = buffalo
        elif  row.Species == 'chickens': x = chicken
        elif  row.Species == 'sheep': x = sheep
        elif  row.Species == 'goats': x = goat
        elif  row.Species == 'pigs': x = pig
        ax1 = plt.subplot( gs[28-r, i] )
        plt.imshow(x, alpha = 0.8)
        plt.axis('off')
        r = r + 1

plt.savefig( '/content/drive/My Drive/figures/livestock/icons.png', dpi = 700, bbox_inches = 'tight', pad_inches = 0,  transparent = True )

In [ ]:
plt.imshow(pig)
plt.axis('off')

In [ ]:
px.colors.qualitative.swatches()



---

# Reference

---



In [ ]:
source = pd.DataFrame([
      {'country': 'Great Britain', 'animal': 'cattle'},
      {'country': 'Great Britain', 'animal': 'cattle'},
      {'country': 'Great Britain', 'animal': 'cattle'},
      {'country': 'Great Britain', 'animal': 'pigs'},
      {'country': 'Great Britain', 'animal': 'pigs'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'Great Britain', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'cattle'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'pigs'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'},
      {'country': 'United States', 'animal': 'sheep'}
    ])

domains = ['person', 'cattle', 'pigs', 'sheep']

shape_scale = alt.Scale(
    domain = domains,
    range = [
        'M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 -0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 -0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 -0.6 -0.4 -0.6z',
        'M4 -2c0 0 0.9 -0.7 1.1 -0.8c0.1 -0.1 -0.1 0.5 -0.3 0.7c-0.2 0.2 1.1 1.1 1.1 1.2c0 0.2 -0.2 0.8 -0.4 0.7c-0.1 0 -0.8 -0.3 -1.3 -0.2c-0.5 0.1 -1.3 1.6 -1.5 2c-0.3 0.4 -0.6 0.4 -0.6 0.4c0 0.1 0.3 1.7 0.4 1.8c0.1 0.1 -0.4 0.1 -0.5 0c0 0 -0.6 -1.9 -0.6 -1.9c-0.1 0 -0.3 -0.1 -0.3 -0.1c0 0.1 -0.5 1.4 -0.4 1.6c0.1 0.2 0.1 0.3 0.1 0.3c0 0 -0.4 0 -0.4 0c0 0 -0.2 -0.1 -0.1 -0.3c0 -0.2 0.3 -1.7 0.3 -1.7c0 0 -2.8 -0.9 -2.9 -0.8c-0.2 0.1 -0.4 0.6 -0.4 1c0 0.4 0.5 1.9 0.5 1.9l-0.5 0l-0.6 -2l0 -0.6c0 0 -1 0.8 -1 1c0 0.2 -0.2 1.3 -0.2 1.3c0 0 0.3 0.3 0.2 0.3c0 0 -0.5 0 -0.5 0c0 0 -0.2 -0.2 -0.1 -0.4c0 -0.1 0.2 -1.6 0.2 -1.6c0 0 0.5 -0.4 0.5 -0.5c0 -0.1 0 -2.7 -0.2 -2.7c-0.1 0 -0.4 2 -0.4 2c0 0 0 0.2 -0.2 0.5c-0.1 0.4 -0.2 1.1 -0.2 1.1c0 0 -0.2 -0.1 -0.2 -0.2c0 -0.1 -0.1 -0.7 0 -0.7c0.1 -0.1 0.3 -0.8 0.4 -1.4c0 -0.6 0.2 -1.3 0.4 -1.5c0.1 -0.2 0.6 -0.4 0.6 -0.4z',
        'M1.2 -2c0 0 0.7 0 1.2 0.5c0.5 0.5 0.4 0.6 0.5 0.6c0.1 0 0.7 0 0.8 0.1c0.1 0 0.2 0.2 0.2 0.2c0 0 -0.6 0.2 -0.6 0.3c0 0.1 0.4 0.9 0.6 0.9c0.1 0 0.6 0 0.6 0.1c0 0.1 0 0.7 -0.1 0.7c-0.1 0 -1.2 0.4 -1.5 0.5c-0.3 0.1 -1.1 0.5 -1.1 0.7c-0.1 0.2 0.4 1.2 0.4 1.2l-0.4 0c0 0 -0.4 -0.8 -0.4 -0.9c0 -0.1 -0.1 -0.3 -0.1 -0.3l-0.2 0l-0.5 1.3l-0.4 0c0 0 -0.1 -0.4 0 -0.6c0.1 -0.1 0.3 -0.6 0.3 -0.7c0 0 -0.8 0 -1.5 -0.1c-0.7 -0.1 -1.2 -0.3 -1.2 -0.2c0 0.1 -0.4 0.6 -0.5 0.6c0 0 0.3 0.9 0.3 0.9l-0.4 0c0 0 -0.4 -0.5 -0.4 -0.6c0 -0.1 -0.2 -0.6 -0.2 -0.5c0 0 -0.4 0.4 -0.6 0.4c-0.2 0.1 -0.4 0.1 -0.4 0.1c0 0 -0.1 0.6 -0.1 0.6l-0.5 0l0 -1c0 0 0.5 -0.4 0.5 -0.5c0 -0.1 -0.7 -1.2 -0.6 -1.4c0.1 -0.1 0.1 -1.1 0.1 -1.1c0 0 -0.2 0.1 -0.2 0.1c0 0 0 0.9 0 1c0 0.1 -0.2 0.3 -0.3 0.3c-0.1 0 0 -0.5 0 -0.9c0 -0.4 0 -0.4 0.2 -0.6c0.2 -0.2 0.6 -0.3 0.8 -0.8c0.3 -0.5 1 -0.6 1 -0.6z',
        'M-4.1 -0.5c0.2 0 0.2 0.2 0.5 0.2c0.3 0 0.3 -0.2 0.5 -0.2c0.2 0 0.2 0.2 0.4 0.2c0.2 0 0.2 -0.2 0.5 -0.2c0.2 0 0.2 0.2 0.4 0.2c0.2 0 0.2 -0.2 0.4 -0.2c0.1 0 0.2 0.2 0.4 0.1c0.2 0 0.2 -0.2 0.4 -0.3c0.1 0 0.1 -0.1 0.4 0c0.3 0 0.3 -0.4 0.6 -0.4c0.3 0 0.6 -0.3 0.7 -0.2c0.1 0.1 1.4 1 1.3 1.4c-0.1 0.4 -0.3 0.3 -0.4 0.3c-0.1 0 -0.5 -0.4 -0.7 -0.2c-0.3 0.2 -0.1 0.4 -0.2 0.6c-0.1 0.1 -0.2 0.2 -0.3 0.4c0 0.2 0.1 0.3 0 0.5c-0.1 0.2 -0.3 0.2 -0.3 0.5c0 0.3 -0.2 0.3 -0.3 0.6c-0.1 0.2 0 0.3 -0.1 0.5c-0.1 0.2 -0.1 0.2 -0.2 0.3c-0.1 0.1 0.3 1.1 0.3 1.1l-0.3 0c0 0 -0.3 -0.9 -0.3 -1c0 -0.1 -0.1 -0.2 -0.3 -0.2c-0.2 0 -0.3 0.1 -0.4 0.4c0 0.3 -0.2 0.8 -0.2 0.8l-0.3 0l0.3 -1c0 0 0.1 -0.6 -0.2 -0.5c-0.3 0.1 -0.2 -0.1 -0.4 -0.1c-0.2 -0.1 -0.3 0.1 -0.4 0c-0.2 -0.1 -0.3 0.1 -0.5 0c-0.2 -0.1 -0.1 0 -0.3 0.3c-0.2 0.3 -0.4 0.3 -0.4 0.3l0.2 1.1l-0.3 0l-0.2 -1.1c0 0 -0.4 -0.6 -0.5 -0.4c-0.1 0.3 -0.1 0.4 -0.3 0.4c-0.1 -0.1 -0.2 1.1 -0.2 1.1l-0.3 0l0.2 -1.1c0 0 -0.3 -0.1 -0.3 -0.5c0 -0.3 0.1 -0.5 0.1 -0.7c0.1 -0.2 -0.1 -1 -0.2 -1.1c-0.1 -0.2 -0.2 -0.8 -0.2 -0.8c0 0 -0.1 -0.5 0.4 -0.8z'
    ]
)

color_scale = alt.Scale(
    domain = domains,
    range = ['rgb(162,160,152)', 'rgb(194,81,64)', 'rgb(93,93,93)', 'rgb(91,131,149)']
)

alt.Chart(source).mark_point(filled=True, opacity=1, size=100).encode(
    alt.X('x:O', axis=None),
    alt.Y('animal:O', axis=None),
    alt.Row('country:N', header=alt.Header(title='')),
    alt.Shape('animal:N', legend=None, scale=shape_scale),
    alt.Color('animal:N', legend=None, scale=color_scale),
).transform_window(
    x='rank()',
    groupby=['country', 'animal']
).properties(width=550, height=140)

In [ ]:
cow = 'U+1F402'
buffalo = 'U+1F403'
sheep = 'U+1F411'
goat = 'U+1F410'
pig = 'U+1F416'
chicken = 'U+1F413'

In [ ]:
alt.Chart(source).mark_text(size=45, baseline='middle').encode(
    alt.X('x:O', axis=None),
    alt.Y('animal:O', axis=None),
    alt.Row('country:N', header=alt.Header(title='')),
    alt.Text('emoji:N')
).transform_calculate(
    emoji="{'cattle': '\U0001F402', 'pigs': '\U0001F416', 'sheep': '\U0001F411' }[datum.animal]"
).transform_window(
    x='rank()',
    groupby=['country', 'animal']
).properties(width=550, height=140)

In [ ]:
shape_scale = alt.Scale(
    domain = domains,
    range = [
        'M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 -0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 -0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 -0.6 -0.4 -0.6z',
        'M4 -2c0 0 0.9 -0.7 1.1 -0.8c0.1 -0.1 -0.1 0.5 -0.3 0.7c-0.2 0.2 1.1 1.1 1.1 1.2c0 0.2 -0.2 0.8 -0.4 0.7c-0.1 0 -0.8 -0.3 -1.3 -0.2c-0.5 0.1 -1.3 1.6 -1.5 2c-0.3 0.4 -0.6 0.4 -0.6 0.4c0 0.1 0.3 1.7 0.4 1.8c0.1 0.1 -0.4 0.1 -0.5 0c0 0 -0.6 -1.9 -0.6 -1.9c-0.1 0 -0.3 -0.1 -0.3 -0.1c0 0.1 -0.5 1.4 -0.4 1.6c0.1 0.2 0.1 0.3 0.1 0.3c0 0 -0.4 0 -0.4 0c0 0 -0.2 -0.1 -0.1 -0.3c0 -0.2 0.3 -1.7 0.3 -1.7c0 0 -2.8 -0.9 -2.9 -0.8c-0.2 0.1 -0.4 0.6 -0.4 1c0 0.4 0.5 1.9 0.5 1.9l-0.5 0l-0.6 -2l0 -0.6c0 0 -1 0.8 -1 1c0 0.2 -0.2 1.3 -0.2 1.3c0 0 0.3 0.3 0.2 0.3c0 0 -0.5 0 -0.5 0c0 0 -0.2 -0.2 -0.1 -0.4c0 -0.1 0.2 -1.6 0.2 -1.6c0 0 0.5 -0.4 0.5 -0.5c0 -0.1 0 -2.7 -0.2 -2.7c-0.1 0 -0.4 2 -0.4 2c0 0 0 0.2 -0.2 0.5c-0.1 0.4 -0.2 1.1 -0.2 1.1c0 0 -0.2 -0.1 -0.2 -0.2c0 -0.1 -0.1 -0.7 0 -0.7c0.1 -0.1 0.3 -0.8 0.4 -1.4c0 -0.6 0.2 -1.3 0.4 -1.5c0.1 -0.2 0.6 -0.4 0.6 -0.4z',
        'M1.2 -2c0 0 0.7 0 1.2 0.5c0.5 0.5 0.4 0.6 0.5 0.6c0.1 0 0.7 0 0.8 0.1c0.1 0 0.2 0.2 0.2 0.2c0 0 -0.6 0.2 -0.6 0.3c0 0.1 0.4 0.9 0.6 0.9c0.1 0 0.6 0 0.6 0.1c0 0.1 0 0.7 -0.1 0.7c-0.1 0 -1.2 0.4 -1.5 0.5c-0.3 0.1 -1.1 0.5 -1.1 0.7c-0.1 0.2 0.4 1.2 0.4 1.2l-0.4 0c0 0 -0.4 -0.8 -0.4 -0.9c0 -0.1 -0.1 -0.3 -0.1 -0.3l-0.2 0l-0.5 1.3l-0.4 0c0 0 -0.1 -0.4 0 -0.6c0.1 -0.1 0.3 -0.6 0.3 -0.7c0 0 -0.8 0 -1.5 -0.1c-0.7 -0.1 -1.2 -0.3 -1.2 -0.2c0 0.1 -0.4 0.6 -0.5 0.6c0 0 0.3 0.9 0.3 0.9l-0.4 0c0 0 -0.4 -0.5 -0.4 -0.6c0 -0.1 -0.2 -0.6 -0.2 -0.5c0 0 -0.4 0.4 -0.6 0.4c-0.2 0.1 -0.4 0.1 -0.4 0.1c0 0 -0.1 0.6 -0.1 0.6l-0.5 0l0 -1c0 0 0.5 -0.4 0.5 -0.5c0 -0.1 -0.7 -1.2 -0.6 -1.4c0.1 -0.1 0.1 -1.1 0.1 -1.1c0 0 -0.2 0.1 -0.2 0.1c0 0 0 0.9 0 1c0 0.1 -0.2 0.3 -0.3 0.3c-0.1 0 0 -0.5 0 -0.9c0 -0.4 0 -0.4 0.2 -0.6c0.2 -0.2 0.6 -0.3 0.8 -0.8c0.3 -0.5 1 -0.6 1 -0.6z',
        'M-4.1 -0.5c0.2 0 0.2 0.2 0.5 0.2c0.3 0 0.3 -0.2 0.5 -0.2c0.2 0 0.2 0.2 0.4 0.2c0.2 0 0.2 -0.2 0.5 -0.2c0.2 0 0.2 0.2 0.4 0.2c0.2 0 0.2 -0.2 0.4 -0.2c0.1 0 0.2 0.2 0.4 0.1c0.2 0 0.2 -0.2 0.4 -0.3c0.1 0 0.1 -0.1 0.4 0c0.3 0 0.3 -0.4 0.6 -0.4c0.3 0 0.6 -0.3 0.7 -0.2c0.1 0.1 1.4 1 1.3 1.4c-0.1 0.4 -0.3 0.3 -0.4 0.3c-0.1 0 -0.5 -0.4 -0.7 -0.2c-0.3 0.2 -0.1 0.4 -0.2 0.6c-0.1 0.1 -0.2 0.2 -0.3 0.4c0 0.2 0.1 0.3 0 0.5c-0.1 0.2 -0.3 0.2 -0.3 0.5c0 0.3 -0.2 0.3 -0.3 0.6c-0.1 0.2 0 0.3 -0.1 0.5c-0.1 0.2 -0.1 0.2 -0.2 0.3c-0.1 0.1 0.3 1.1 0.3 1.1l-0.3 0c0 0 -0.3 -0.9 -0.3 -1c0 -0.1 -0.1 -0.2 -0.3 -0.2c-0.2 0 -0.3 0.1 -0.4 0.4c0 0.3 -0.2 0.8 -0.2 0.8l-0.3 0l0.3 -1c0 0 0.1 -0.6 -0.2 -0.5c-0.3 0.1 -0.2 -0.1 -0.4 -0.1c-0.2 -0.1 -0.3 0.1 -0.4 0c-0.2 -0.1 -0.3 0.1 -0.5 0c-0.2 -0.1 -0.1 0 -0.3 0.3c-0.2 0.3 -0.4 0.3 -0.4 0.3l0.2 1.1l-0.3 0l-0.2 -1.1c0 0 -0.4 -0.6 -0.5 -0.4c-0.1 0.3 -0.1 0.4 -0.3 0.4c-0.1 -0.1 -0.2 1.1 -0.2 1.1l-0.3 0l0.2 -1.1c0 0 -0.3 -0.1 -0.3 -0.5c0 -0.3 0.1 -0.5 0.1 -0.7c0.1 -0.2 -0.1 -1 -0.2 -1.1c-0.1 -0.2 -0.2 -0.8 -0.2 -0.8c0 0 -0.1 -0.5 0.4 -0.8z'
    ]
)

In [ ]:
!pip install svgpathtools
from svgpathtools import svg2paths

In [ ]:
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/buffalo.svg')
buffalo = attributes[0]['d']
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/chicken.svg')
chicken = attributes[0]['d']
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/cow.svg')
cow = attributes[0]['d']
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/goat.svg')
goat = attributes[0]['d']
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/sheep.svg')
sheep = attributes[0]['d']
paths, attributes = svg2paths('/content/drive/My Drive/figures/livestock/pig.svg')
pig = attributes[0]['d']
animal_shapes = [buffalo, chicken, cow, goat, pig, sheep ]

In [ ]:
animal_shapes = [buffalo, chicken, cow, goat, pig, sheep ]

# Climate data

In [ ]:
pigs
def rothfusz90(Ta, RH): return 2.049 * (1.8 * Ta + 32) + (10.143 * RH) - (0.224 * (Ta * 1.8 + 32) * RH) - (0.0068 * ((Ta * 1.8 + 32)**2)) - (0.055 * (RH**2)) + (0.0012 * ((Ta * 1.8 + 32)**2) * RH) + (0.0009 * (Ta * 1.8 + 32) * (RH**2)) - (0.00000199 * ((Ta * 1.8 + 32)**2)*(RH**2)) - 42.379

In [ ]:
%%capture
!pip install --upgrade zarr gcsfs cftime nc-time-axis
import fsspec, zarr, gcsfs; gcs = gcsfs.GCSFileSystem( token = 'anon' )

df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
df = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & source_id == 'IPSL-CM6A-LR' & table_id == 'day'" )
df
ds = xr.open_zarr( gcs.get_mapper( df.zstore.values[0] ), consolidated = True )

In [ ]:
ds = xr.open_zarr( fsspec.get_mapper('gcs://pangeo-era5/reanalysis/spatial-analysis'),
                  consolidated = True, chunks = {'time': -1, 'latitude':10, 'longitude':10}  )
ds

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.quantile( quants, dim = 'time' ).to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/1984_d2m_90up_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.d2m.slice(time = slice('1984-01-01T00:00:00', '1984-12-31T23:00:00')).quantile( quants, dim = 'time' ).to_netcdf(path)

In [ ]:
path = '/content/drive/My Drive/data/livestock/t2m_90_quantile.nc4'
quants = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99]
ds.t2m.sel( latitude = slice(90,87.5) ).resample( time = '1D' ).max().quantile( quants, dim = 'time' ).to_netcdf(path)

### CMIP6

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )
df = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & source_id == 'IPSL-CM6A-LR' & table_id == 'day'" )
df
ds = xr.open_zarr( gcs.get_mapper( df.zstore.values[0] ), consolidated = True )

In [ ]:
ds = xr.open_zarr( gcs.get_mapper( df.zstore.values[0] ), consolidated = True )

In [ ]:
import dask.array as da

In [ ]:
from dask.diagnostics import ProgressBar
a = da.random.normal(size=(10000, 10000), chunks=(1000, 1000))
res = a.dot(a.T).mean(axis=0)

with ProgressBar():
    out = res.compute()

In [ ]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")


In [ ]:
!pip install intake

In [ ]:
import requests

In [ ]:
    if    RH == False: RH  = 100*( np.exp( ( 17.625 * Tdp ) / ( 243.04 + Tdp ) ) / np.exp( ( 17.625 * Ta ) / ( 243.04 + Ta ) ) )
    elif Tdp == False: Tdp = 243.04 * (np.log(RH/100) + ( ( 17.625 * Ta ) / ( 243.04 + Ta ) ) ) / ( 17.625 - np.log(RH/100) - ( ( 17.625 * Ta ) / ( 243.04 + Ta ) ) )
    elif
    Twb = Ta * np.arctan( 0.151977 * (RH + 8.313659)**(1/2) ) + np.arctan(Ta+ RH) - np.arctan(RH - 1.676331) + 0.00391838 * (RH)**(3/2) * np.arctan(0.023101 * RH) - 4.686035

In [ ]:
e0 = 6.113 # saturation vapor pressure in hPa
# e0 and Pressure have to be in same units
c_water = 5423 # L/R for water in Kelvin
c_ice = 6139 # L/R for ice in Kelvin
T0 = 273.15 # Kelvin

if latent == 'water' or latent == 'Water':
    self.c = self.c_water    # using c for water

else:
    self.c= self.c_ice       # using c_ice for ice, clear state

# saturation vapor not required, uncomment to calculate it (units in hPa becuase of e0)
#sat_vapor = self.e0 * np.exp((self.c * (self.temp -self.T0))/(self.temp * self.T0))

#calculating specific humidity, q directly from dew point temperature
#using equation 4.24, Pg 96 Practical Meteorolgy (Roland Stull)
( np.log(q * 26.299) / 5423 ) * T = (Tdp - T)/(Tdp * T)

RH = s p / (0.622 e_s(T))
e0 = 6.113 # saturation vapor pressure in hPa
# e0 and Pressure have to be in same units
c_water = 5423 # L/R for water in Kelvin
c_ice = 6139 # L/R for ice in Kelvin
T0 = 273.15 # Kelvin

if latent == 'water' or latent == 'Water':
    self.c = self.c_water    # using c for water

else:
    self.c= self.c_ice       # using c_ice for ice, clear state

# saturation vapor not required, uncomment to calculate it (units in hPa becuase of e0)
#sat_vapor = self.e0 * np.exp((self.c * (self.temp -self.T0))/(self.temp * self.T0))

#calculating specific humidity, q directly from dew point temperature
#using equation 4.24, Pg 96 Practical Meteorolgy (Roland Stull)
( np.log(q * 26.299) / 5423 ) * T = (Tdp - T)/(Tdp * T)

RH = s p / (0.622 e_s(T))


In [ ]:
def THI(Ta, Tdp, huss, spec ):
    Ta = np.atleast_1d( Ta )
    temp  = Ta * units.degC
    press = np.array( 1e5 ) * units.hectopascal

    if huss == False:
        dew = np.atleast_1d( Tdp ) * units.degC
        RH  = np.asarray( mpcalc.relative_humidity_from_dewpoint( temp , dew ) )
        Twb = np.asarray( mpcalc.wet_bulb_temperature( press, temp, dew ) )

    elif Tdp == False:
        SH = np.atleast_1d( huss ) * units.dimensionless
        dew =  mpcalc.dewpoint_from_specific_humidity( SH, temp, press )
        RH  = np.asarray( mpcalc.relative_humidity_from_specific_humidity(SH, temp, press ) )
        Twb = np.asarray( mpcalc.wet_bulb_temperature( press, temp, dew ) )
        Tdp = np.asarray( dew )

    this = np.ma.zeros( ( Ta.shape[0], 14) )

    def thom59(Ta, Twb):    return 0.72 * (Ta + Twb) + 47
    def bianca62a(Ta, Twb): return (0.35 * Ta + 0.65 * Twb) * 1.8 + 32
    def bianca62b(Ta, Twb): return (0.15 * Ta + 0.85 * Twb) * 1.8 + 32
    def kibler64(Ta, RH):   return 1.8 * Ta - (1 - RH/100) * (Ta - 14.3) + 32
    def ingram65(Ta, Twb):  return 0.63 * Twb + 1.17 * Ta + 32
    def roller69(Ta, Twb):  return 0.45 * Twb + 1.35 * Ta + 32
    def nrc71a(Ta, RH):     return (1.8 * Ta + 32) - ((0.55 - 0.0055 * RH) * (1.8 * Ta - 26.8))
    def nrc71b(Ta, Tdp):    return 1.8 * (0.55 * Ta + 0.2 * Tdp)  + 49.5
    def nrc71c(Ta, Twb):    return (Ta + Twb) * 0.72 + 40.6
    def noaa76(Ta, RH):     return (1.8 * Ta + 32) - (0.55 - 0.0055* RH )* (1.8 * Ta - 26)
    def yousef85(Ta, Tdp):  return Ta + (0.36 * Tdp) + 41.2
    def rothfusz90(Ta, RH): return (2.049 * (Ta * 1.8 + 32)) + (10.143 * RH) - (0.224 * (Ta * 1.8 + 32) * RH) - (0.0068 * ((Ta * 1.8 + 32)**2)) - (0.055 * (RH**2)) + (0.0012 * ((Ta * 1.8 + 32)**2) * RH) + (0.0009 * (Ta * 1.8 + 32) * (RH**2)) - (0.00000199 * ((Ta * 1.8 + 32)**2)*(RH**2)) - 42.379
    def hahn99(Ta, RH):     return (0.8 * Ta) + ((RH/100) * (Ta - 14.4)) + 46.4
    def berman16(Ta, RH):   return 3.43 + (1.058 * Ta) - (0.293 * RH) + (0.0164 * Ta * RH) + 35.7

    this[:,0] = thom59(Ta, Twb)
    this[:,1] = bianca62a(Ta, Twb)
    this[:,2] = bianca62b(Ta, Twb)
    this[:,3] = kibler64(Ta, RH)
    this[:,4] = ingram65(Ta, Twb)
    this[:,5] = roller69(Ta, Twb)
    this[:,6] = nrc71a(Ta, RH)
    this[:,7] = nrc71b(Ta, Tdp)
    this[:,8] = nrc71c(Ta, Twb)
    this[:,9] = noaa76(Ta, RH)
    this[:,10] = yousef85(Ta, Tdp)
    this[:,11] = rothfusz90(Ta, RH)
    this[:,12] = hahn99(Ta, RH)
    this[:,13] = berman16(Ta, RH)

    this = np.ma.masked_outside( np.nan_to_num( this ), 1, 200 )
    this = np.ma.compress_rows( this )

    return this